## update: 2022-03-12
pytorch 1.10 에서 trainloader 의 Iterator 가 reset 되지 않았던 문제가 있었던 것이 확실함...
pytroch 1.11.0 으로 올라오면서 해당 에러가 수정된 것 같아서 trainloader 를 그때그때 새롭게 부르는 부분을 수정해야 함 

seq2seq 에서 Attention 구현은 꽤나 복잡합니다. 

* seq2seq 에서 Attention 을 어떻게 계산하는지에 대해서는 아래의 youtube 를 보시면 기본개념에 대해서 이해하기 수월합니다.

https://www.youtube.com/watch?v=WsQLdu2JMgI

* 네트웍의 연결구조가 코드를 봐서 쉽게 이해가 안될수도 있습니다. attention network 구조가 단순히 encoder/decoder 구조로만 이루어진 seq2seq 이랑 비교해서 상당히 복잡합니다.  계산된 attention weight 값이 rnn cell 의 input embedding 에 합쳐지기도 하고, rnn 에서 나온 출력값이 최종 출력을 위해서 linear-softmax 로 들어갈때도 같이 concatenate 됩니다. 다음장에 나오는 transformer 에서도 내부적으로 많은 residual connection 을 사용합니다. 

https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html


* 아래의 코드는 seq2seq with attention 구현으로 가장 널리 알려지고, 복제된(?) bentrevett 의 구현을 이용했습니다. 

https://github.com/bentrevett/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb

위 구현체에서 dimension 을 이해하는데 약간의 어려움을 주는 부분인 encoder 에서의 bidirectional-RNN 을 unidirectional-RNN 으로 만들었습니다.    encoder 와 decoder 와 hidden dimension 을 갖도록 만들었다고 보시면 되어요.  ( 원논문에 encoder 부분만 bidirectional 로 정의되어 있어서 따라한 것 같습니다.)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch.nn.functional as F

import spacy
import numpy as np

import random
import math
import time

/home/chester/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
tokenizer_en = get_tokenizer('spacy', language='en_core_web_sm')
tokenizer_de = get_tokenizer('spacy', language='de_core_news_sm')

In [4]:

def yield_tokens_en(data_iter):
    for data in data_iter:
        yield tokenizer_en(data[1].rstrip('\n'))
        
def yield_tokens_de(data_iter):
    for data in data_iter:
        yield tokenizer_de(data[0].rstrip('\n'))
        
    

In [5]:
train, valid, test = Multi30k('./.data', split=('train', 'valid', 'test'), language_pair=('de','en'))
en_vocab = build_vocab_from_iterator(yield_tokens_en(train), min_freq=2, specials=['<unk>', '<pad>', '<sos>', '<eos>'])
de_vocab = build_vocab_from_iterator(yield_tokens_de(train), min_freq=2, specials=['<unk>', '<pad>', '<sos>', '<eos>'])

/home/chester/.local/lib/python3.8/site-packages/torch/utils/data/datapipes/utils/common.py:24: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn(
/home/chester/.local/lib/python3.8/site-packages/torch/utils/data/datapipes/iter/selecting.py:54: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn("Lambda function is not supported for pickle, please use "
/home/chester/.local/lib/python3.8/site-packages/torch/utils/data/datapipes/iter/combining.py:180: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [6]:
en_vocab.set_default_index(en_vocab['<unk>'])
de_vocab.set_default_index(de_vocab['<unk>'])

In [7]:
print(len(en_vocab))
print(len(de_vocab))

6191
8014


In [8]:
PAD_IDX = en_vocab['<pad>']
SOS_IDX = en_vocab['<sos>']
EOS_IDX = en_vocab['<eos>']
UNK_IDX = en_vocab['<unk>']

print(UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX)

0 1 2 3


In [9]:
en_text_pipeline = lambda x : en_vocab(tokenizer_en(x))
de_text_pipeline = lambda x : de_vocab(tokenizer_de(x))

In [10]:
def collate_batch(batch):
    de_list, en_list = [], []
    for data in batch:
        processed_de = torch.tensor( de_text_pipeline(data[0]), dtype=torch.int64)
        processed_en = torch.tensor( en_text_pipeline(data[1]), dtype=torch.int64)
        de_list.append( torch.cat( [torch.tensor([SOS_IDX]), processed_de, torch.tensor([EOS_IDX])], dim=0))
        en_list.append( torch.cat( [torch.tensor([SOS_IDX]), processed_en, torch.tensor([EOS_IDX])], dim=0))
    de_list = pad_sequence( de_list, padding_value=PAD_IDX)
    en_list = pad_sequence( en_list, padding_value=PAD_IDX)
    return de_list, en_list


In [11]:
train, valid, test = Multi30k('./.data', split=('train', 'valid', 'test'), language_pair=('de','en'))

BATCH_SIZE = 64

train_loader = DataLoader( train, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
valid_loader = DataLoader(valid, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

여기서부터 Encoder, Attention, Decoder 를 한번 만들어보도록 하자.
주의할점은 지난 10과에서 구현했던 seq2seq 와 마찬가지로 Encoder 에서 RNN 계산을 할때는 src 문장의 모든 sequence 를 (seq_len, batch_len, hidden_dim ) 으로 넣었다면 Decoder 를 계산할때는 매 스텝의 RNN output 을 가지고 attention 을 계산하고, 바로 전 output 의 출력값을 input 으로 넣어줘야 하기 때문에 Decoder 에 입력되는 tensor 의 dimension 이 (1, batch_len, hidden_dim ) 이 된다는 것이다. 이 부분에 혼돈이 없도록 하는데 주의가 필요하다.

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim)
        self.fc = nn.Linear(enc_hid_dim, dec_hid_dim)
        self.dropout= nn.Dropout(dropout)
        
    def forward(self, src):
        # src = [src_len, N ]
        
        embedded = self.dropout(self.embedding(src))
        # embedded : [ src_len, N, emb_dim ]
        
        outputs, hidden = self.rnn(embedded)
        # outputs : [ src_len, N, enc_hid_dim ] 
        # hidden : [ n_layers * n_direction, N, enc_hid_dim ]
        
        last_hidden_for_attention = torch.tanh(self.fc(hidden.squeeze(0)))
        # last_hidden_for_attention = [ N, dec_hid_dim ] 
        
        # outputs : [ src_len, N, enc_hid_dim ] 
        # last_hidden_for_attention  : [ N, dec_hid_dim ] 
        
        return outputs, last_hidden_for_attention


In [14]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear( enc_hid_dim + dec_hid_dim, dec_hid_dim )
        self.v = nn.Linear( dec_hid_dim, 1, bias=False)
        
    def forward(self, hidden, encoder_outputs):
        
        # hidden : [ batch_size, dec_hid_dim ]
        # encoder_outputs : [ src_len, N, enc_hid_dim ]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        # decoder 의 hidden ouput 을 encoder 의 src_len 갯수만큼 늘림 
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        # hidden : [ N, src_len, dec_hid_dim ] 이 되는 것에 주의!

        encoder_outputs = encoder_outputs.permute(1,0,2) 
        # hidden 의 dimension 인 [ N , src_len, enc_hid_dim ] 으로 맞추어 준다....  통상적으로 [ src_len, N, enc_hid_dim ] 으로 되어 있음

        concat = torch.cat((hidden, encoder_outputs), dim=2)
        
        energy = torch.tanh(self.attn(concat))
        # hidden 과 encoder_outs 에 있는 hidden 값을 concatenate 해서 넣어줌, 
        
        # energy : [ N, src_len, dec_hid_dim ] 
        
        attention = self.v(energy).squeeze(2) 
        # hid_dim 으로 되어 있는 값을 하나의 값으로 weighted sum 을 해줌...
        # 그리고 나서 맨 마지막에 있는 불필요한 dimension 을 떼어줘서 softmax 에 넣을 준비를 함 
        
        alpha = F.softmax( attention, dim=1 )
        
        return alpha


In [15]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim) 
        self.rnn = nn.GRU( enc_hid_dim + emb_dim, dec_hid_dim )
        self.fc_out = nn.Linear( enc_hid_dim + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        
        # input : [ N ]  ( encoder 와 달리 하나의 단어씩 들어오는 것에 주의 : class Seq2seq 의 forward() 를 참고
        # hidden : [ N, dec_hid_dim ]
        # encoder_outputs : [ src_len, N, enc_hid_dim ] 
        
        input = input.unsqueeze(0)
        # input : [ 1, N ] ... 이렇게 차원을 맞추어 주어야 pytorch nn.Embedding 등 함수에 넣을 수 있다. 
        
        embedded = self.dropout(self.embedding(input))
        # embedded : [ 1, N, emb_dim ]
        
        a = self.attention(hidden, encoder_outputs)
        # a : [ N, src_len ] .....   SOFTMAX 값이 들어가 있음
        
        a = a.unsqueeze(1)  
        # a : [ N, 1, src_len ]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        # encoder_outputs : [ N, src_len, enc_hid_dim ] 

        weighted = torch.bmm(a, encoder_outputs)
        # weighted : [ N, 1, enc_hid_dim ] 
        
        weighted = weighted.permute(1,0,2)
        # weighted : [ 1, N, enc_hid_dim ] 으로 다시 변경함
        
        rnn_input = torch.cat( (embedded, weighted), dim=2 )
        # rnn_input [ 1, N, enc_hid_dim + emb_dim ] 
        
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        # output : [ 1, N, dec_hid_dim ] 
        # hidden : [ 1, N, dec_hid_dim ] 
        
        # thus
        assert(output==hidden).all()
        
        # final outcome 을 내는 linear layer 에 넣기 위해서 trg_len 에 해당하는 0-dimension 을 모두 제거함 
        # 즉, [ 1, N, dec_hid_dim ] --> [ N, dec_hid_dim ] 

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out( torch.cat( ( output, weighted, embedded), dim=1))
        
        # prediction : [ N, output_dim ] .... 이제 여기에 argmax 를 붙히면 
        return prediction, hidden.squeeze(0)
    

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5 ):
        # src : [ src_len, N ]
        # trg : [ trg_len, N ]
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # 전체 output 을 저장할 placehoder 만들어두기 
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        encoder_outputs, hidden = self.encoder(src)
        
        # decoder 에 넣을 첫번때 input 은 SOS token 이 되어야 함
        input = trg[0,:] 
        # input : [ N ] 
        
        for t in range(1, trg_len):  
            
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            
            top1 = output.argmax(1)
            
            input = trg[t] if teacher_force else top1
            
        return outputs

In [17]:
INPUT_DIM = len(de_vocab)
OUTPUT_DIM = len(en_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT,attn)

model = Seq2Seq(enc, dec, device).to(device)



In [18]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(8014, 256)
    (rnn): GRU(256, 512)
    (fc): Linear(in_features=512, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1024, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(6191, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=6191, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 15,506,991 trainable parameters


In [20]:
optimizer = optim.Adam(model.parameters())


In [21]:
PAD_IDX = en_vocab['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [22]:

def train( model, iterator, optimizer, criterion, clip):
    
    model.train()
    epoch_loss = 0 
    iter_len = 0 
    
    for i, batch in enumerate(iterator):
        src = batch[0].to(device)
        trg = batch[1].to(device)
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        # trg = [target_length , batch_len]
        # output = [target_len, batch_len, output_dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        iter_len += 1
    return epoch_loss /iter_len
        
   

In [23]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    iter_len = 0 
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0].to(device)
            trg = batch[1].to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
            iter_len += 1
        
    return epoch_loss /iter_len

In [24]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
N_EPOCHS = 15
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 57s
	Train Loss: 4.689 | Train PPL: 108.711
	 Val. Loss: 4.614 |  Val. PPL: 100.876
Epoch: 02 | Time: 0m 54s
	Train Loss: 3.708 | Train PPL:  40.789
	 Val. Loss: 3.974 |  Val. PPL:  53.185
Epoch: 03 | Time: 0m 55s
	Train Loss: 3.034 | Train PPL:  20.785
	 Val. Loss: 3.585 |  Val. PPL:  36.039
Epoch: 04 | Time: 0m 55s
	Train Loss: 2.573 | Train PPL:  13.109
	 Val. Loss: 3.386 |  Val. PPL:  29.551
Epoch: 05 | Time: 0m 55s
	Train Loss: 2.242 | Train PPL:   9.412
	 Val. Loss: 3.426 |  Val. PPL:  30.740
Epoch: 06 | Time: 0m 55s
	Train Loss: 1.987 | Train PPL:   7.293
	 Val. Loss: 3.402 |  Val. PPL:  30.036
Epoch: 07 | Time: 0m 55s
	Train Loss: 1.784 | Train PPL:   5.953
	 Val. Loss: 3.427 |  Val. PPL:  30.793
Epoch: 08 | Time: 0m 55s
	Train Loss: 1.635 | Train PPL:   5.129
	 Val. Loss: 3.440 |  Val. PPL:  31.202
Epoch: 09 | Time: 0m 55s
	Train Loss: 1.520 | Train PPL:   4.570
	 Val. Loss: 3.526 |  Val. PPL:  33.999
Epoch: 10 | Time: 0m 55s
	Train Loss: 1.411 | Train PPL

In [26]:
model.load_state_dict(torch.load('tut1-model.pt'))


test_loss = evaluate(model, test_loader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.345 | Test PPL:  28.370 |
